In [1]:
import dgl
import numpy as np
import torch

from dgl.data.utils import load_graphs
from ogb.nodeproppred import DglNodePropPredDataset

dataset = DglNodePropPredDataset(name = 'ogbn-mag')

split_idx = dataset.get_idx_split()
train_idx, valid_idx, test_idx = split_idx["train"], split_idx["valid"], split_idx["test"]

def node_level_subsampling(g, list_of_nodes, node_numbers):
    subsample_data = {}
    if len(list_of_nodes) == 0:
        raise Error('list of nodes are empty')
    
    for node_type in list_of_nodes:
        subsample_data[node_type]=g.nodes(node_type)[:node_numbers]
    
    return dgl.node_subgraph(g,subsample_data)
    

def create_mask_from_idx(idx,total_nodes):
    mask_array = np.zeros(total_nodes,dtype=bool)
    mask_array[idx]=True
    return mask_array

def load_mag_data():
    graph = load_graphs('./mag_mp.bin')
    _, label = dataset[0]
    g = graph[0][0]
    print(g)
    features = g.ndata['feat']['paper']
    labels = label['paper']
    mask = torch.BoolTensor(create_mask_from_idx(train_idx['paper'], g.num_nodes('paper')))
   
    return g, features, labels, mask

Using backend: pytorch


In [2]:
import dgl.nn as dglnn
import torch.nn as nn
import torch.nn.functional as F

In [3]:
# Define a Heterograph Conv model

import dgl.function as fn

class RGCN(nn.Module):
    def __init__(self, in_feats, hid_feats, out_feats, rel_names):
        super().__init__()

        self.conv1 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(in_feats, hid_feats)
            for rel in rel_names}, aggregate='sum')
        self.conv2 = dglnn.HeteroGraphConv({
            rel: dglnn.GraphConv(hid_feats, out_feats)
            for rel in rel_names}, aggregate='sum')

    def forward(self, graph, inputs):
        # inputs are features of nodes
        h = self.conv1(graph, inputs)
        h = {k: F.relu(v) for k, v in h.items()}
        h = self.conv2(graph, h)
        return h

In [4]:
g, features, labels, mask = load_mag_data()
# g = node_level_subsampling(g,['paper','author','field_of_study','institution'],10000)
# g = g.e
print(g)

Graph(num_nodes={'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389},
      num_edges={('author', 'ai', 'institution'): 1043998, ('author', 'ap', 'paper'): 7145660, ('field_of_study', 'fp', 'paper'): 7505078, ('institution', 'ia', 'author'): 1043998, ('paper', 'pa', 'author'): 7145660, ('paper', 'pf', 'field_of_study'): 7505078, ('paper', 'pp', 'paper'): 5416271},
      metagraph=[('author', 'institution', 'ai'), ('author', 'paper', 'ap'), ('institution', 'author', 'ia'), ('paper', 'author', 'pa'), ('paper', 'field_of_study', 'pf'), ('paper', 'paper', 'pp'), ('field_of_study', 'paper', 'fp')])
Graph(num_nodes={'author': 1134649, 'field_of_study': 59965, 'institution': 8740, 'paper': 736389},
      num_edges={('author', 'ai', 'institution'): 1043998, ('author', 'ap', 'paper'): 7145660, ('field_of_study', 'fp', 'paper'): 7505078, ('institution', 'ia', 'author'): 1043998, ('paper', 'pa', 'author'): 7145660, ('paper', 'pf', 'field_of_study'): 7505078, ('paper'

In [7]:
model = RGCN(128, 20, 349,g.etypes)
# model = RGCN(len(features), 128, 349, ['cites'])
# paper_feats = features[:10000]
paper_feats = features
author_feats = torch.zeros([g.num_nodes('author'), 128])
fos_feats = torch.zeros([g.num_nodes('field_of_study'), 128])
institute_feats = torch.zeros([g.num_nodes('institution'), 128])
train_mask = mask
print(paper_feats.shape)
print(author_feats.shape)
node_features = {'paper': paper_feats, 'author':author_feats, 'field_of_study':fos_feats, 'institution': institute_feats}
# h_dict = model(g, {'paper': paper_feats})
g.etypes

torch.Size([736389, 128])
torch.Size([1134649, 128])


['ai', 'ap', 'fp', 'ia', 'pa', 'pf', 'pp']

In [8]:
labels

tensor([[246],
        [131],
        [189],
        ...,
        [266],
        [289],
        [  1]])

In [6]:
# Create the model. The output has three logits for three classes.
# model = HeteroRGCN(g, 500, 128, 349)

opt = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

best_val_acc = 0
best_test_acc = 0

for epoch in range(1):
    model.train()
    # forward propagation by using all nodes and extracting the user embeddings
    logits = model(g, node_features)['paper']
    loss = F.cross_entropy(logits[train_idx['paper']], labels[train_idx['paper'].squeeze(1)])

    pred = logits.argmax(1)
#     train_acc = (pred[train_idx] == labels[train_idx]).float().mean()
#     val_acc = (pred[val_idx] == labels[val_idx]).float().mean()
#     test_acc = (pred[test_idx] == labels[test_idx]).float().mean()

#     if best_val_acc < val_acc:
#         best_val_acc = val_acc
#         best_test_acc = test_acc

    opt.zero_grad()
    loss.backward()
    opt.step()

#     if epoch % 5 == 0:
#         print('Loss %.4f, Train Acc %.4f, Val Acc %.4f (Best %.4f), Test Acc %.4f (Best %.4f)' % (
#             loss.item(),
#             train_acc.item(),
#             val_acc.item(),
#             best_val_acc.item(),
#             test_acc.item(),
#             best_test_acc.item(),
#         ))



RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:73] data. DefaultCPUAllocator: not enough memory: you tried to allocate 1027999044 bytes. Buy new RAM!

In [ ]:
print(min(loss_array))